## Data manpulation widgets toy examples

In [ ]:
from example_widgets import ProcessWidget, PlotWidget, fake_load, setup_code_hiding, allowed_dimensions, filepath_converter
import scipp as sc
import functools

# Workaround for the fact that you can't raise directly in tertiary expressions.
def raise_(ex):
    raise ex

setup_code_hiding()

In [ ]:
plot_widget = PlotWidget(globals())

### Data Loading
Currently creates some example data rather than loading a file. The created file is a histogrammed time of flight data.

In [ ]:
data_creation = ProcessWidget(globals(), fake_load, 'Load', {'filepath': filepath_converter}, {'filepath': 'run number or file name'})
data_creation.subscribe(plot_widget)
data_creation

### Data conversion
This converts one physical dimension to another the current conversions support in scipp.neutron are:
* tof -> dspacing, wavelength, E
* d-spacing -> tof
* wavelength -> Q, tof
* E -> tof
* Q -> wavelength

In [ ]:
dim_converter = lambda dim : dim if dim in allowed_dimensions else raise_(ValueError(
    f'{dim} not an allowed dimension. Supported dimensions are {list(dimesion_to_unit.keys())}'))
data_converter = lambda name : globals()[name] if (name in globals()) else raise_(ValueError(
    f'{name} does not exist in notebook.'))
data_options = lambda : [
            key for key, item in globals().items()
            if isinstance(item, (sc.DataArray, sc.Dataset, sc.Variable))
        ]

data_conversion = ProcessWidget(globals(), 
                                  sc.neutron.convert, 'Convert', 
                                  {'data': data_converter,
                                   'from' : dim_converter, 'to': dim_converter},
                                   options={'from' : allowed_dimensions,
                                            'to': allowed_dimensions,
                                            'data': data_options})
data_conversion.subscribe(plot_widget)
data_conversion

Can also specify just some inputs of a function graphically.

In [ ]:
# Workaround because python doesn't like shadowing of from
kwargs = {'from':'tof'}
convert_tof = functools.partial(sc.neutron.convert, **kwargs)
data_conversion_tof = ProcessWidget(globals(), 
                                  convert_tof, 'Convert from tof', 
                                  {'data': data_converter,
                                   'to': dim_converter}, 
                                   options={'to': allowed_dimensions,
                                            'data': data_options})
data_conversion_tof.subscribe(plot_widget)
data_conversion_tof

### Data plotting
Plots the data aquired by evaluating the expression entered.

In [ ]:
plot_widget